In [ ]:
# -*- coding:utf-8 -*-
% matplotlib inline
from __future__ import print_function
from matplotlib import pyplot as plt
import matplotlib.pylab as pylab
import os
import cv2
import numpy as np

In [ ]:
MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15

def alignImages(im1, im2):
 
  # Convert images to grayscale
  im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
  im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
   
  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(MAX_FEATURES)
  keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
   
  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = matcher.match(descriptors1, descriptors2, None)
   
  # Sort matches by score
  matches.sort(key=lambda x: x.distance, reverse=False)
 
  # Remove not so good matches
  numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
  matches = matches[:numGoodMatches]
 
  # Draw top matches
  imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  cv2.imwrite("matches.jpg", imMatches)
   
  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)
 
  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt
   
  # Find homography
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
 
  # Use homography
  height, width, channels = im2.shape
  im1Reg = cv2.warpPerspective(im1, h, (width, height))
   
  return im1Reg, h

## Read reference image

In [ ]:
refFilename = "./pictures/anchor.jpg"
print("Reading reference image : ", refFilename)
imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)
plt.imshow(imReference)
plt.show()

## Read image to be aligned

In [ ]:
imFilename = "./pictures/target.jpg"
print("Reading image to align : ", imFilename);  
im = cv2.imread(imFilename, cv2.IMREAD_COLOR)
plt.imshow(im)
plt.show()

## Registered image will be resotred in imReg.

In [ ]:
print("Aligning images ...") 
# The estimated homography will be stored in h. 
imReg, h = alignImages(im, imReference)
plt.imshow(imReg)
plt.show()

# Write aligned image to disk. 
print("Saving aligned image : ", "./pictures/aligned.jpg"); 
cv2.imwrite("./pictures/aligned.jpg", imReg)

# Print estimated homography
print("Estimated homography : \n",  h)

## SIF图像配准

In [ ]:
def sift_kp(image):
    gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    sift=cv2.xfeatures2d.SIFT_create()
    kp,des = sift.detectAndCompute(image,None)
    kp_image = cv2.drawKeypoints(gray_image,kp,None)
    return kp_image,kp,des

In [ ]:
def get_good_match(des1,des2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2) #des1为模板图，des2为匹配图
    matches = sorted(matches,key=lambda x:x[0].distance/x[1].distance)
    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append(m)
    return good

In [ ]:
def siftImageAlignment(img1,img2):
   _,kp1,des1 = sift_kp(img1)
   _,kp2,des2 = sift_kp(img2)
   goodMatch = get_good_match(des1,des2)
   if len(goodMatch) > 4:
       ptsA= np.float32([kp1[m.queryIdx].pt for m in goodMatch]).reshape(-1, 1, 2)
       ptsB = np.float32([kp2[m.trainIdx].pt for m in goodMatch]).reshape(-1, 1, 2)
       ransacReprojThreshold = 4
       H, status =cv2.findHomography(ptsA,ptsB,cv2.RANSAC,ransacReprojThreshold);
       imgOut = cv2.warpPerspective(img2, H, (img1.shape[1],img1.shape[0]),flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
   return imgOut,H,status

## 配准

In [ ]:
# img1 = cv2.imread(r"./pictures/anchor.jpg")
# img2 = cv2.imread(r"./pictures/target.jpg")
 
# _,kp1,des1 = sift_kp(img1)
# _,kp2,des2 = sift_kp(img2)
# goodMatch = get_good_match(des1,des2)
 
# img3 = cv2.drawMatches(img1, kp1, img2, kp2, goodMatch[:5], None, flags=2)
# #----or----
# #goodMatch = np.expand_dims(goodMatch,1)
# #img3 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, goodMatch[:5], None, flags=2)
 
# plt.imshow('img',img3)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

## 形状匹配

In [ ]:
#性状匹配
img1 = cv2.imread("./pictures/anchor.jpg")
img2 = cv2.imread("./pictures/target.jpg")

gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
ret, thresh1 = cv2.threshold(gray1, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
plt.imshow(thresh1)
plt.show()

gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
ret, thresh2 = cv2.threshold(gray2, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
plt.imshow(thresh2)
plt.show()

copyImage1, contours1, hierarchy1 = cv2.findContours(thresh1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
copyImage2, contours2, hierarchy2 = cv2.findContours(thresh2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnt1 = contours1[0]

cnt = contours2[0]
#找到性状的重心
M = cv2.moments(cnt)
cx = int(M['m10']/M['m00'])
cy = int(M['m01']/M['m00'])
print(cx, cy)
cv2.circle(img2, (cx, cy), 5, (255,255,255), -1)
#性状匹配
ret = cv2.matchShapes(cnt1, cnt, 1, 0.0)
print(ret)
plt.imshow(gray2)
plt.show()

## 阈值分割

In [ ]:
def segmentation(path):
    files = os.listdir(path)
    for name in files:
        img_name = os.path.join(path, name)
        img = cv2.imread(img_name)
        
        gray1 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, thresh1 = cv2.threshold(gray1, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
        plt.imshow(thresh1, plt.cm.gray)
        plt.show()

In [ ]:
segmentation("./pictures/")

## 图像去燥+阈值分割

In [ ]:
img = cv2.imread("./pictures/anchor.jpg")
# BGR -> RGB
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
# Denoising
dst = cv2.fastNlMeansDenoisingColored(img, None,10,10,7,21)
plt.subplot(221), plt.imshow(img)
plt.subplot(222), plt.imshow(dst)

# Gray
gray = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
plt.subplot(223), plt.imshow(gray, plt.cm.gray)

# Bin
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
plt.subplot(224), plt.imshow(thresh, plt.cm.gray)
